In [1]:
import os

if "_executed_once" not in get_ipython().user_ns:
    print("This cell runs only once per kernel restart.")
    get_ipython().user_ns["_executed_once"] = True

    import nest_asyncio

    nest_asyncio.apply()

    os.chdir("../")


This cell runs only once per kernel restart.


In [ ]:
from pydantic_ai import Agent
from pydantic_ai.providers.ollama import OllamaProvider
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.mcp import MCPServerStreamableHTTP
import logfire

logfire.configure()
logfire.instrument_pydantic_ai()


# ollama_model = OpenAIChatModel(
#     model_name="qwen3:8b",
#     provider=OllamaProvider(base_url="http://localhost:11434/v1"),
# )

ollama_model = OpenAIChatModel(
    model_name="gpt-oss:120b-cloud",
    provider=OllamaProvider(base_url="https://ollama.com/v1",
                            api_key=os.getenv("OLLAMA_API_KEY")),
)

doc_server = MCPServerStreamableHTTP("http://localhost:8002/mcp")
neo4j_server = MCPServerStreamableHTTP("http://localhost:8001/api/mcp")
system_prompt= """You are a helpful assistant providing information based one the Internet Yellow Pages (IYP, a knowdledge graph about the Internet).
Use available tools to query the database or retrieve informations about the databse documentation. Query IYP cypher query if needed to reply to the user. If the user's request cannot be answered by IYP, simply say that you don't know"""
agent = Agent(
    ollama_model, toolsets=[doc_server, neo4j_server], system_prompt=system_prompt
)

# result = agent.run_sync(
#     "Give me the best description of the dataset related to tranco website ranking"
# )

result = agent.run_sync(
    "Give me the best description of the dataset related to tranco website ranking"
)
print(result.output)


Logfire project URL: ]8;id=324961;https://logfire-us.pydantic.dev/justin-loye/iyp-mcp\https://logfire-us.pydantic.dev/justin-loye/iyp-mcp]8;;\

07:49:02.272 agent run
07:49:02.333   chat gpt-oss:120b-cloud
07:49:03.788   running 1 tool
07:49:03.788     running tool: list_iyp_datasets
07:49:03.857   chat gpt-oss:120b-cloud
07:49:08.655   running 1 tool
07:49:08.655     running tool: get_resource
07:49:08.668   chat gpt-oss:120b-cloud
**Dataset:** **Tranco – “Tranco list”** (`tranco.top1m`)

**Description (from the IYP documentation)**  

> The Tranco list is a research‑oriented top‑sites ranking hardened against manipulation. It **combines the rankings of several source lists** to produce a daily list that is based on data of the past 30 days.  
> IYP uses this data to create and annotate `DomainName` nodes.

**Key points**

- **Purpose:** Provides a stable, manipulation‑resistant ranking of the most popular web sites.  
- **Source:** Aggregates multiple existing ranking sources (see the Tranco methodology at https://tranco-list.eu/methodology).  
- **Coverage:** Daily list of the top 1 million domains, reflecting the last 30 d

In [3]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="You are a helpful assistant providing information based one the Internet Yellow Pages (IYP, a knowdledge graph about the Internet).\nUse available tools to query the database or retrieve informations about the databse documentation. Query IYP cypher query if needed to reply to the user. If the user's request cannot be answered by IYP, simply say that you don't know", timestamp=datetime.datetime(2025, 11, 27, 7, 49, 2, 325287, tzinfo=datetime.timezone.utc)), UserPromptPart(content='Give me the best description of the dataset related to tranco website ranking', timestamp=datetime.datetime(2025, 11, 27, 7, 49, 2, 325292, tzinfo=datetime.timezone.utc))], run_id='edbf49a3-12e7-4c7c-8c75-29809de143cd'),
 ModelResponse(parts=[ThinkingPart(content='We need to give best description of the dataset related to tranco website ranking. Use IYP dataset info. Need to list datasets then get resource for the dataset. So first list datasets to find name matc

In [4]:
from typing import Any
from datetime import datetime
import json


def pretty_print_messages(messages: list[Any], verbose: bool = False) -> None:
    """
    Pretty print a list of Pydantic AI messages.

    Args:
        messages: List of ModelRequest and ModelResponse objects
        verbose: If True, show additional metadata like timestamps, usage, etc.
    """
    for i, msg in enumerate(messages):
        print(f"\n{'=' * 80}")
        print(f"Message {i + 1}")
        print(f"{'=' * 80}")

        # Determine message type
        if hasattr(msg, "kind"):
            if msg.kind == "request":
                _print_request(msg, verbose)
            elif msg.kind == "response":
                _print_response(msg, verbose)
        else:
            print(f"Unknown message type: {type(msg)}")


def _print_request(msg: Any, verbose: bool) -> None:
    """Print a ModelRequest message."""
    print("📤 REQUEST")

    if verbose and hasattr(msg, "instructions") and msg.instructions:
        print(f"\nInstructions: {msg.instructions}")

    print("\nParts:")
    for j, part in enumerate(msg.parts):
        _print_request_part(part, j, verbose)


def _print_response(msg: Any, verbose: bool) -> None:
    """Print a ModelResponse message."""
    print("📥 RESPONSE")

    if verbose:
        if hasattr(msg, "model_name") and msg.model_name:
            print(f"Model: {msg.model_name}")
        if hasattr(msg, "provider_name") and msg.provider_name:
            print(f"Provider: {msg.provider_name}")
        if hasattr(msg, "timestamp"):
            print(f"Timestamp: {msg.timestamp}")
        if hasattr(msg, "usage"):
            print(f"Usage: {msg.usage}")
        if hasattr(msg, "provider_request_id") and msg.provider_request_id:
            print(f"Request ID: {msg.provider_request_id}")
        if hasattr(msg, "provider_details") and msg.provider_details:
            print(f"Provider Details: {msg.provider_details}")
        if hasattr(msg, "finish_reason") and msg.finish_reason:
            print(f"Finish Reason: {msg.finish_reason}")

    print("\nParts:")
    for j, part in enumerate(msg.parts):
        _print_response_part(part, j, verbose)


def _print_request_part(part: Any, index: int, verbose: bool) -> None:
    """Print a single ModelRequestPart."""
    part_kind = getattr(part, "part_kind", "unknown")

    print(f"\n  [{index}] {_get_part_icon(part_kind)} {part_kind.upper()}")

    if part_kind == "system-prompt":
        print(f"      Content: {_truncate(part.content)}")
        if verbose:
            if hasattr(part, "timestamp"):
                print(f"      Timestamp: {part.timestamp}")
            if hasattr(part, "dynamic_ref") and part.dynamic_ref:
                print(f"      Dynamic Ref: {part.dynamic_ref}")

    elif part_kind == "user-prompt":
        content = part.content
        if isinstance(content, str):
            print(f"      Content: {_truncate(content)}")
        elif isinstance(content, list):
            print(f"      Content (multipart):")
            for item in content:
                _print_content_item(item)
        if verbose and hasattr(part, "timestamp"):
            print(f"      Timestamp: {part.timestamp}")

    elif part_kind == "tool-return":
        print(f"      Tool: {part.tool_name}")
        print(f"      Tool Call ID: {part.tool_call_id}")
        print(f"      Content: {_format_json(part.content, indent=8)}")
        if verbose:
            if hasattr(part, "timestamp"):
                print(f"      Timestamp: {part.timestamp}")
            if hasattr(part, "metadata") and part.metadata:
                print(f"      Metadata: {part.metadata}")

    elif part_kind == "retry-prompt":
        print(f"      Tool: {part.tool_name or 'N/A'}")
        print(f"      Tool Call ID: {part.tool_call_id or 'N/A'}")
        if isinstance(part.content, str):
            print(f"      Content: {_truncate(part.content)}")
        else:
            print(f"      Content (errors): {len(part.content)} validation errors")
            if verbose:
                for error in part.content[:3]:  # Show first 3 errors
                    print(f"        - {error}")
        if verbose and hasattr(part, "timestamp"):
            print(f"      Timestamp: {part.timestamp}")


def _print_response_part(part: Any, index: int, verbose: bool) -> None:
    """Print a single ModelResponsePart."""
    part_kind = getattr(part, "part_kind", "unknown")

    print(f"\n  [{index}] {_get_part_icon(part_kind)} {part_kind.upper()}")

    if part_kind == "text":
        print(f"      Content: {_truncate(part.content)}")

    elif part_kind == "thinking":
        print(f"      Content: {_truncate(part.content)}")
        if verbose:
            if hasattr(part, "id") and part.id:
                print(f"      ID: {part.id}")
            if hasattr(part, "signature") and part.signature:
                print(f"      Signature: {part.signature}")

    elif part_kind == "tool-call":
        print(f"      Tool: {part.tool_name}")
        print(f"      Tool Call ID: {part.tool_call_id}")
        args = part.args
        if isinstance(args, str):
            print(f"      Args (JSON): {_truncate(args)}")
        else:
            print(f"      Args: {_format_json(args, indent=8)}")

    elif part_kind == "builtin-tool-call":
        print(f"      Tool: {part.tool_name} (builtin)")
        print(f"      Provider: {part.provider_name or 'N/A'}")
        print(f"      Tool Call ID: {part.tool_call_id}")
        args = part.args
        if isinstance(args, str):
            print(f"      Args (JSON): {_truncate(args)}")
        else:
            print(f"      Args: {_format_json(args, indent=8)}")

    elif part_kind == "builtin-tool-return":
        print(f"      Tool: {part.tool_name} (builtin)")
        print(f"      Provider: {part.provider_name or 'N/A'}")
        print(f"      Tool Call ID: {part.tool_call_id}")
        print(f"      Content: {_format_json(part.content, indent=8)}")
        if verbose:
            if hasattr(part, "timestamp"):
                print(f"      Timestamp: {part.timestamp}")
            if hasattr(part, "metadata") and part.metadata:
                print(f"      Metadata: {part.metadata}")


def _print_content_item(item: Any) -> None:
    """Print a content item from multipart user prompt."""
    if isinstance(item, str):
        print(f"        - Text: {_truncate(item)}")
    elif hasattr(item, "kind"):
        kind = item.kind
        if kind == "image-url":
            print(f"        - Image URL: {item.url}")
        elif kind == "video-url":
            print(f"        - Video URL: {item.url}")
        elif kind == "audio-url":
            print(f"        - Audio URL: {item.url}")
        elif kind == "document-url":
            print(f"        - Document URL: {item.url}")
        elif kind == "binary":
            media = item.media_type
            size = len(item.data) if hasattr(item, "data") else 0
            print(f"        - Binary: {media} ({size} bytes)")
    elif hasattr(item, "type"):
        print(f"        - {item.type}: {item}")
    else:
        print(f"        - Unknown: {type(item)}")


def _get_part_icon(part_kind: str) -> str:
    """Get an emoji icon for the part kind."""
    icons = {
        "system-prompt": "⚙️",
        "user-prompt": "👤",
        "tool-return": "🔧",
        "retry-prompt": "🔄",
        "text": "💬",
        "thinking": "🤔",
        "tool-call": "🛠️",
        "builtin-tool-call": "🏗️",
        "builtin-tool-return": "🏗️",
    }
    return icons.get(part_kind, "❓")


def _truncate(text: str, max_length: int = 10000) -> str:
    """Truncate text to a maximum length."""
    if len(text) <= max_length:
        return text
    return text[:max_length] + "..."


def _format_json(obj: Any, indent: int = 0) -> str:
    """Format an object as JSON with proper indentation."""
    if obj is None:
        return "null"

    try:
        if isinstance(obj, str):
            # Try to parse as JSON if it's a string
            try:
                parsed = json.loads(obj)
                return json.dumps(parsed, indent=2).replace("\n", "\n" + " " * indent)
            except:
                return obj
        else:
            return json.dumps(obj, indent=2, default=str).replace(
                "\n", "\n" + " " * indent
            )
    except:
        return str(obj)


In [5]:
pretty_print_messages(result.all_messages(), verbose=True)


Message 1
📤 REQUEST

Parts:

  [0] ⚙️ SYSTEM-PROMPT
      Content: You are a helpful assistant providing information based one the Internet Yellow Pages (IYP, a knowdledge graph about the Internet).
Use available tools to query the database or retrieve informations about the databse documentation. Query IYP cypher query if needed to reply to the user. If the user's request cannot be answered by IYP, simply say that you don't know
      Timestamp: 2025-11-27 07:49:02.325287+00:00

  [1] 👤 USER-PROMPT
      Content: Give me the best description of the dataset related to tranco website ranking
      Timestamp: 2025-11-27 07:49:02.325292+00:00

Message 2
📥 RESPONSE
Model: gpt-oss:120b
Provider: ollama
Timestamp: 2025-11-27 07:49:03+00:00
Usage: RequestUsage(input_tokens=615, output_tokens=74)
Request ID: chatcmpl-613
Provider Details: {'finish_reason': 'tool_calls'}
Finish Reason: tool_call

Parts:

  [0] 🤔 THINKING
      Content: We need to give best description of the dataset related to

/tmp/ipykernel_2272486/1214004877.py:54: DeprecationWarning: `provider_request_id` is deprecated, use `provider_response_id` instead
  if hasattr(msg, "provider_request_id") and msg.provider_request_id:
/tmp/ipykernel_2272486/1214004877.py:55: DeprecationWarning: `provider_request_id` is deprecated, use `provider_response_id` instead
  print(f"Request ID: {msg.provider_request_id}")


In [6]:
# result2 = agent.run_sync("Run one of the cypher query you found in the documentation.", message_history=result.new_messages())